# Explore here

In [13]:
# Your code here

# Paso 1: Carga del conjunto de datos

import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")

data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [15]:
data.duplicated().sum()

np.int64(630)

In [16]:
data["is_spam"] = data["is_spam"].apply(lambda x: 1 if x else 0).astype(int)
data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [18]:
print(data.shape)
print(f"is_spam: {len(data.loc[data.is_spam == 1])}")
print(f"not_is_spam: {len(data.loc[data.is_spam == 0])}")

(2999, 2)
is_spam: 696
not_is_spam: 2303


In [19]:
data = data.drop_duplicates()
data = data.reset_index(inplace = False, drop = True)
data.shape

(2369, 2)

In [ ]:
# Paso 2: Preprocesa los enlaces

In [ ]:
# Paso 2: Preprocesa los enlaces

